In [247]:
from flow.scenarios import MergeScenario
from flow.controllers import ContinuousRouter, IDMController
from flow.core.params import SumoCarFollowingParams, SumoLaneChangeParams

In [248]:
from flow.core.params import VehicleParams
from flow.controllers import IDMController
from flow.core.params import SumoCarFollowingParams

# create an empty vehicles object
vehicles = VehicleParams()

# add some vehicles to this object of type "human"
vehicles.add(
    veh_id = "human",
    acceleration_controller=(IDMController, {}),
    routing_controller = (ContinuousRouter, {}),
    car_following_params=SumoCarFollowingParams(
        speed_mode="obey_safe_speed",
    ),
    lane_change_params=SumoLaneChangeParams(
        lane_change_mode= "strategic",
    ), num_vehicles = 100)

In [249]:
from flow.core.params import InFlows

inflow = InFlows()

In [250]:
inflow.add(veh_type="human",
           edge="inflow_highway",
           vehs_per_hour=2000,
           departSpeed=10,
           departLane="random")

In [251]:
inflow.add(veh_type="human",
           edge="inflow_merge",
           vehs_per_hour=1000,
           departSpeed=10,
           departLane="random")

In [252]:
from flow.core.params import NetParams

additional_net_params = {
    # length of the merge edge
    "merge_length": 100,
    # length of the highway leading to the merge
    "pre_merge_length": 200,
    # length of the highway past the merge
    "post_merge_length": 100,
    # number of lanes in the merge
    "merge_lanes": 2,
    # number of lanes in the highway
    "highway_lanes": 5,
    # max speed limit of the network
    "speed_limit": 30,
}

# we choose to make the main highway slightly longer
additional_net_params["pre_merge_length"] = 500

net_params = NetParams(inflows=inflow,  # our inflows
                       no_internal_links=False,
                       additional_params=additional_net_params)

In [253]:
from numpy import pi, sin, cos

class customMergeScenario(MergeScenario):
    
    def specify_nodes(self, net_params):
        """See parent class."""
        angle = pi / 4
        wider_angle = (2*pi)/3
        merge = net_params.additional_params["merge_length"]
        premerge = net_params.additional_params["pre_merge_length"]
        postmerge = net_params.additional_params["post_merge_length"]

        nodes = [
            {
                "id": "inflow",
                "x": -100,
                "y": 0
            },
            {
                "id": "outflow",
                "y": 0,
                "x": premerge + postmerge
            },
            {
                "id": "center_1",
                "y": 0,
                "x": premerge,
                "radius": 10
            },
            {
                "id": "center_2",
                "y": 0,
                "x": postmerge,
                "radius" : 10
            }, 
            {
                "id": "bottom_1",
                "x": premerge - merge * cos(angle),
                "y": -merge * sin(angle)
            },
            {
                "id": "bottom_2",
                "x": postmerge - merge * cos(angle),
                "y": -merge * sin(angle)
            },
                        {
                "id": "merge_in",
                "y": -(merge + INFLOW_EDGE_LEN) * sin(angle),
                "x": premerge - (merge + INFLOW_EDGE_LEN) * cos(angle),
            },
            {
                "id": "merge_out",
                "x": postmerge - (merge + INFLOW_EDGE_LEN) * cos(angle),
                "y": -(merge + INFLOW_EDGE_LEN) * sin(angle)
            }
        ]

        return nodes
    
    def specify_edges(self, net_params):
        """See parent class."""
        merge = net_params.additional_params["merge_length"]
        premerge = net_params.additional_params["pre_merge_length"]
        postmerge = net_params.additional_params["post_merge_length"]

        edges = [{
            "id": "inflow_highway_to_center_1",
            "type": "highwayType",
            "from": "inflow_highway",
            "to": "center_1",
            "length": 100
        }, {
            "id": "bottom_1_to_center_1",
            "type": "highwayType",
            "from": "bottom_1",
            "to": "center_1",
            "length": 100
        },{
            "id": "center_1_to_center_2",
            "type": "highwayType",
            "from": "center_1",
            "to": "center_2",
            "length": 100
        },{
            "id": "merge_in_to_bottom_1",
            "type": "highwayType",
            "from": "merge_in",
            "to": "bottom_1",
            "length": 100
        },{
            "id": "center_2_to_bottom_2",
            "type": "highwayType",
            "from": "center_2",
            "to": "bottom_2",
            "length": 100
        },{
            "id": "center_2_to_outflow",
            "type": "highwayType",
            "from": "center_2",
            "to": "outflow",
            "length": 100
        },{
            "id": "bottom_2_to_merge_out",
            "type": "highwayType",
            "from": "bottom_2",
            "to": "merge_out",
            "length": 100
        }]

        return edges
    
    def specify_routes(self, net_params):
        """See parent class."""
        rts = {
            "inflow_highway_to_center_1": ["inflow_highway_to_center_1",
                                           "center_1_to_center_2",
                                           "center_2_to_outflow"],
            
            "center_1_to_center_2" : ["center_1_to_center_2",
                                      "center_2_to_outflow"],
            
            "center_2_to_outflow" : ["center_2_to_outflow"],
            
            "merge_in_to_bottom_1" : ["merge_in_to_bottom_1",
                                     "bottom_1_to_center_1", "center_1_to_center_2"],
            
            "bottom_1_to_center_1" : ["bottom_1_to_center_1", "center_1_to_center_2"]
        }

        return rts
    
    def specify_edge_starts(self):
        """See parent class."""
        premerge = self.net_params.additional_params["pre_merge_length"]
        postmerge = self.net_params.additional_params["post_merge_length"]

        edgestarts = [("inflow_highway_to_center_1", 0),
                      ("bottom_1_to_center_1", 90 + 0.1),
                      ("center_1_to_center_2", 100 + 0.1),
                      ("merge_in_to_bottom_1", 80 + 0.1),
                      ("center_2_to_bottom_2",
                       100 + premerge + postmerge + 22.6),
                      ("center_2_to_outflow",
                       100 + postmerge + 22.6),
                      ("bottom_2_to_merge_out",
                       2 * 100 + premerge + postmerge + 22.7)]

        return edgestarts    

In [254]:
from flow.core.params import SumoParams, EnvParams, InitialConfig
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.core.experiment import Experiment

sumo_params = SumoParams(render=True,
                         sim_step=0.2)

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

initial_config = InitialConfig(spacing="random", perturbation=1)

scenario = customMergeScenario(
    name="merge-example",
    vehicles=vehicles,
    net_params=net_params,
    initial_config=initial_config
)

env = AccelEnv(env_params, sumo_params, scenario)

exp = Experiment(env)

_ = exp.run(1, 10000)

Error during start: [Errno 2] No such file or directory: '/Users/apple/Desktop/Berkeley/Forks/flow/flow/core/kernel/scenario/debug/cfg/merge-example_20190625-1509151561500555.612235.net.xml'
Retrying in 1 seconds...
Error during start: [Errno 2] No such file or directory: '/Users/apple/Desktop/Berkeley/Forks/flow/flow/core/kernel/scenario/debug/cfg/merge-example_20190625-1509151561500555.612235.net.xml'
Retrying in 1 seconds...
Error during start: [Errno 2] No such file or directory: '/Users/apple/Desktop/Berkeley/Forks/flow/flow/core/kernel/scenario/debug/cfg/merge-example_20190625-1509151561500555.612235.net.xml'
Retrying in 1 seconds...
Error during start: [Errno 2] No such file or directory: '/Users/apple/Desktop/Berkeley/Forks/flow/flow/core/kernel/scenario/debug/cfg/merge-example_20190625-1509151561500555.612235.net.xml'
Retrying in 1 seconds...
Error during start: [Errno 2] No such file or directory: '/Users/apple/Desktop/Berkeley/Forks/flow/flow/core/kernel/scenario/debug/cfg/m

TypeError: exceptions must derive from BaseException